In [1]:
import glob
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import pandas as pd

## Functions

In [2]:
def convert_date(date):
    # Remove special characters
    date = re.sub('[^a-z0-9]', " ", date)
    try:
        # ASSUMING dd MMM
        # Extract day
        day_start = re.search('^\d+', date).start()
        day_end = re.search('^\d+', date).end()
        day = date[day_start:day_end]
        if len(day) == 1:
            day = "0"+day
        # Extract month
        date_wo_day = date[day_end:].strip()
        month_start = re.search('^\w+', date_wo_day).start()
        month_end = re.search('^\w+', date_wo_day).end()
        month = date_wo_day[month_start:month_end]
        if "oct" in str(month):
            month = 10
        if "nov" in str(month):
            month = 11
        if "dec" in str(month):
            month = 12

        full_date = day+"."+str(month)+".2019"
        if len(full_date) == 10:
            return full_date
        else:
            return "nil"
    except:
        return "nil"

In [3]:
def convert_time(time, post_time):
    return_time = "nil"
    time_original = str(re.sub(r'[.:-]', '', time)).strip()
    
    # ASSUME NUM PM
    if bool(re.search(r'^\d+ ?pm', time_original)):
        time = time_original.replace("pm", "").strip()
        time = str(time)
        # If 1,2,3,4,5,6,7,8,9
        if len(str(time)) == 1:
            time = int(time)
            if time == 1:
                return_time = "13:00"
            if time == 2:
                return_time = "14:00"
            if time == 3:
                return_time = "15:00"
            if time == 4:
                return_time = "16:00"
            if time == 5:
                return_time = "17:00"
            if time == 6:
                return_time = "18:00"
            if time == 7:
                return_time = "19:00"
            if time == 8:
                return_time = "20:00"
            if time == 9:
                return_time = "21:00"
        # If 10,11,12
        if len(str(time)) == 2:
            time = int(time)
            if time == 10:
                return_time = "22:00"
            if time == 11:
                return_time = "23:00"
            if time == 12:
                return_time = "12:00"
        # If single hour (eg. 1) and minutes (eg. 15)
        if len(str(time)) == 3:
            hour = int(time[0])
            if hour == 1:
                return_time = "13:"+time[1:]
            if hour == 2:
                return_time = "14:"+time[1:]
            if hour == 3:
                return_time = "15:"+time[1:]
            if hour == 4:
                return_time = "16:"+time[1:]
            if hour == 5:
                return_time = "17:"+time[1:]
            if hour == 6:
                return_time = "18:"+time[1:]
            if hour == 7:
                return_time = "19:"+time[1:]
            if hour == 8:
                return_time = "20:"+time[1:]
            if hour == 9:
                return_time = "21:"+time[1:]
        # If double hour (eg. 11) and minutes (eg. 30)
        if len(str(time)) == 4:
            hour = int(time[0:2])
            if hour == 10:
                return_time = "22:"+time[2:]
            if hour == 11:
                return_time = "23:"+time[2:]
            if hour == 12:
                return_time = "12:"+time[2:]
    
    # ASSUME NUM AM
    if bool(re.search(r'^\d+ ?am', time_original)):
        time = time_original.replace("am", "").strip()
        time = str(time)
        # If 1,2,3,4,5,6,7,8,9
        if len(str(time)) == 1:
            time = int(time)
            if time == 1:
                return_time = "01:00"
            if time == 2:
                return_time = "02:00"
            if time == 3:
                return_time = "03:00"
            if time == 4:
                return_time = "04:00"
            if time == 5:
                return_time = "05:00"
            if time == 6:
                return_time = "06:00"
            if time == 7:
                return_time = "07:00"
            if time == 8:
                return_time = "08:00"
            if time == 9:
                return_time = "09:00"
        # If 10,11,12
        if len(str(time)) == 2:
            time = int(time)
            if time == 10:
                return_time = "10:00"
            if time == 11:
                return_time = "11:00"
            if time == 12:
                return_time = "00:00"
        # If single hour (eg. 1) and minutes (eg. 15)
        if len(str(time)) == 3:
            hour = int(time[0])
            if hour == 1:
                return_time = "01:"+time[1:]
            if hour == 2:
                return_time = "02:"+time[1:]
            if hour == 3:
                return_time = "03:"+time[1:]
            if hour == 4:
                return_time = "04:"+time[1:]
            if hour == 5:
                return_time = "05:"+time[1:]
            if hour == 6:
                return_time = "06:"+time[1:]
            if hour == 7:
                return_time = "07:"+time[1:]
            if hour == 8:
                return_time = "08:"+time[1:]
            if hour == 9:
                return_time = "09:"+time[1:]
        # If double hour (eg. 11) and minutes (eg. 30)
        if len(str(time)) == 4:
            hour = int(time[0:2])
            if hour == 10:
                return_time = "10:"+time[2:]
            if hour == 11:
                return_time = "11:"+time[2:]
            if hour == 12:
                return_time = "00:"+time[2:]
    
    # ASSUME NO PM OR AM
    if bool(re.search(r'^\d+$', time_original)):
        time = str(time_original).strip()
        # If single digit
        if len(time) == 1:
            time = int(time)
            post_hour = post_time[:2]
            if int(post_hour) >= 12:
                if time == 1:
                    return_time = "13:00"
                if time == 2:
                    return_time = "14:00"
                if time == 3:
                    return_time = "15:00"
                if time == 4:
                    return_time = "16:00"
                if time == 5:
                    return_time = "17:00"
                if time == 6:
                    return_time = "18:00"
                if time == 7:
                    return_time = "19:00"
                if time == 8:
                    return_time = "20:00"
                if time == 9:
                    return_time = "21:00"
            else:
                if time == 1:
                    return_time = "01:00"
                if time == 2:
                    return_time = "02:00"
                if time == 3:
                    return_time = "03:00"
                if time == 4:
                    return_time = "04:00"
                if time == 5:
                    return_time = "05:00"
                if time == 6:
                    return_time = "06:00"
                if time == 7:
                    return_time = "07:00"
                if time == 8:
                    return_time = "08:00"
                if time == 9:
                    return_time = "09:00"
        # If double digits
        if len(str(time)) == 2:
            time = int(time)
            post_hour = post_time[:2]
            if int(post_hour) >= 12:
                if time == 10:
                    return_time = "22:00"
                if time == 11:
                    return_time = "23:00"
                if time == 12:
                    return_time = "12:00"
            else:
                if time == 10:
                    return_time = "10:00"
                if time == 11:
                    return_time = "11:00"
                if time == 12:
                    return_time = "00:00"    
        # If three digtis
        if len(str(time)) == 3:
            hour = int(str(time)[0])
            post_hour = post_time[:2]
            if int(post_hour) >= 12:
                if hour == 1:
                    return_time = "13:"+time[1:]
                if hour == 2:
                    return_time = "14:"+time[1:]
                if hour == 3:
                    return_time = "15:"+time[1:]
                if hour == 4:
                    return_time = "16:"+time[1:]
                if hour == 5:
                    return_time = "17:"+time[1:]
                if hour == 6:
                    return_time = "18:"+time[1:]
                if hour == 7:
                    return_time = "19:"+time[1:]
                if hour == 8:
                    return_time = "20:"+time[1:]
                if hour == 9:
                    return_time = "21:"+time[1:]
            else:
                if hour == 1:
                    return_time = "01:"+time[1:]
                if hour == 2:
                    return_time = "02:"+time[1:]
                if hour == 3:
                    return_time = "03:"+time[1:]
                if hour == 4:
                    return_time = "04:"+time[1:]
                if hour == 5:
                    return_time = "05:"+time[1:]
                if hour == 6:
                    return_time = "06:"+time[1:]
                if hour == 7:
                    return_time = "07:"+time[1:]
                if hour == 8:
                    return_time = "08:"+time[1:]
                if hour == 9:
                    return_time = "09:"+time[1:]
        # If four digtis
        if len(str(time)) == 4:
            hour = int(str(time)[0:2])
            post_hour = post_time[:2]
            if int(post_hour) >= 12:
                if hour >= 12:
                    return_time = str(hour)+":"+str(time)[2:]
                if hour == 10:
                    return_time = "22:"+time[2:]
                if hour == 11:
                    return_time = "23:"+time[2:]
                if hour == 0:
                    return_time = "00:"+time[2:]
            else:
                if hour <= 12:
                    if len(str(hour)) == 1:
                        return_time = "0"+str(hour)+":"+str(time)[2:]
                    else:
                        return_time = str(hour)+":"+str(time)[2:]
                if hour == 0:
                    return_time = "00:"+time[2:]
                if hour == 10:
                    return_time = "10:"+time[2:]
                if hour == 11:
                    return_time = "11:"+time[2:]
                if hour == 12:
                    return_time = "00:"+time[2:]
        
    return return_time

## Process Data

### Read Files

In [11]:
html_files_old = glob.glob("./telegram_chat_export_beginning_19sep/*.html")
print("Number of files: ", len(html_files_old))
print(html_files_old[:10])
print()

html_files_new = glob.glob("./telegram_chat_export_13oct_6dec/*.html")
print("Number of files: ", len(html_files_new))
print(html_files_new[:10])

Number of files:  257
['./telegram_chat_export_beginning_19sep\\messages.html', './telegram_chat_export_beginning_19sep\\messages10.html', './telegram_chat_export_beginning_19sep\\messages100.html', './telegram_chat_export_beginning_19sep\\messages101.html', './telegram_chat_export_beginning_19sep\\messages102.html', './telegram_chat_export_beginning_19sep\\messages103.html', './telegram_chat_export_beginning_19sep\\messages104.html', './telegram_chat_export_beginning_19sep\\messages105.html', './telegram_chat_export_beginning_19sep\\messages106.html', './telegram_chat_export_beginning_19sep\\messages107.html']

Number of files:  148
['./telegram_chat_export_13oct_6dec\\messages.html', './telegram_chat_export_13oct_6dec\\messages10.html', './telegram_chat_export_13oct_6dec\\messages100.html', './telegram_chat_export_13oct_6dec\\messages101.html', './telegram_chat_export_13oct_6dec\\messages102.html', './telegram_chat_export_13oct_6dec\\messages103.html', './telegram_chat_export_13oct_6

In [12]:
html_files = [*html_files_old, *html_files_new]
print("Total html files: ", len(html_files))

Total html files:  405


### Extract Data from Files

In [13]:
messages_list = []
total_messages = 0

for file in tqdm(html_files):    
    f = open(file, "r", encoding='utf-8')
    chat = f.read()
    soup = BeautifulSoup(chat)
    messages = soup.findAll("div", {"class": "message default clearfix"})
    total_messages += len(messages)

    for message in messages:
        # Get user
        user = message.find("div", {"class": "from_name"}).text.replace("\n", "").strip()
        #print(user)
        # Get post date and time
        post_datetime = message.find("div", {"class": "pull_right date details"}).get("title")
        post_date = post_datetime.split(" ")[0]
        post_time = post_datetime.split(" ")[1]
        #print(post_date)
        #print(post_time)
        # GET MESSAGE
        raw_message = message.find("div", {"class": "text"})
        # Extract text from message
        raw_message = str(raw_message).replace("</div>", "").replace('<div class="text">', "").replace("<strong>", "").replace("</strong>", "").replace("<u>", "").replace("</u>", "").strip().lower()
        # Tokenize text
        raw_message_tokens = re.sub(r'(<br/> ?){2,}', '<br/>', raw_message).split("<br/>")

        # Variable flag for actual ride request
        is_request = False
        # Get user type
        if(bool(re.search(r'hitchers? looking', raw_message_tokens[0]))):
            user_type = "hitcher"
            is_request = True
        elif(bool(re.search(r'driver? looking', raw_message_tokens[0]))):
            user_type = "driver"
            is_request = True
        else:
            user_type = "nil"

        # PROCESS MESSAGE CONTENT IF IT IS ACTUALLY A REQUEST
        if(is_request):
            # Blank place holder
            pick_up = "nil"
            drop_off = "nil"
            request_date = "nil"
            request_time = "nil"
            number_of_pax = "nil"
            # ------------------

            # Extract pick up location assuming it is always the second token
            try:
                pick_up_tokens = str(raw_message_tokens[1]).split(":")
            except:
                pass
            try:
                if "pick" in pick_up_tokens[0]:
                    pick_up = pick_up_tokens[1].strip()
            except:
                pick_up_tokens = str(raw_message_tokens[1]).split(" ")
                if "pick" in pick_up_tokens[0]:
                    pick_up = " ".join(pick_up_tokens[1:])
                else:
                    pass

            # Extract drop off location assuming it is always the third token
            try:
                drop_off_tokens = str(raw_message_tokens[2]).split(":")
            except:
                pass
            try:
                if "drop" in drop_off_tokens[0]:
                    drop_off = drop_off_tokens[1].strip()
            except:
                drop_off_tokens = str(raw_message_tokens[2]).split(" ")
                if "drop" in drop_off_tokens[0]:
                    drop_off = " ".join(drop_off_tokens[1:])
                else:
                    pass

            # Extract date assuming it is always the fourth token
            try:
                date_tokens = str(raw_message_tokens[3]).split(":")
            except:
                pass
            try:
                if "date" in date_tokens[0]:
                    request_date = date_tokens[1].strip()
            except:
                date_tokens = str(raw_message_tokens[3]).split(" ")
                if "date" in date_tokens[0]:
                    request_date = " ".join(date_tokens[1:])
                else:
                    pass
            if "today" in request_date or "tdy" in request_date:
                request_date = post_date
            elif "tmr" in request_date or "tomorrow" in request_date:
                tomorrow_day = post_date.split(".")[0]
                tomorrow_day = int(tomorrow_day)+1
                if len(str(tomorrow_day)) == 1:
                    tomorrow_day = "0"+str(tomorrow_day)
                request_date = str(tomorrow_day)+"."+post_date.split(".")[1]+".2019"
            elif request_date == "nil":
                pass
            else:
                request_date = convert_date(request_date)

            # Extract time assuming it is always the fifth token
            try:
                time_raw = str(raw_message_tokens[4])
            except:
                pass
            try:
                time = time_raw.replace("time:", "").replace("time :", "").strip()
                time = re.sub(r'[^a-z0-9.-:]', '', time).strip()
                if "now" in time:
                    request_time = post_time[:5]
                else:
                    request_time = convert_time(str(time), post_time)
            except:
                pass

            # Extract number of pax assuming it is always the sixth token
            try:
                if "pax" in raw_message_tokens[5]:
                    raw_pax = str(raw_message_tokens[5]).replace("pax:", "").replace("pax", "").replace("pax :", "").replace("pax : ", "").strip()
                    raw_pax = raw_pax.replace(":", "").strip()
                    raw_pax = re.sub(r'\(.*\)', '', raw_pax)
                    raw_pax = re.sub(r'[^a-z0-9- ]', '', raw_pax)
                    raw_pax = raw_pax.strip()
                    if len(raw_pax) == 1:
                        number_of_pax = raw_pax
                    elif len(raw_pax) == 3:
                        number_of_pax = raw_pax
                    else:
                        if bool(re.search(r'^\d+ - \d+$', raw_pax)):
                            number_of_pax = raw_pax.replace(" ", "")
                        if bool(re.search(r'^\d+ to \d+$', raw_pax)):
                            number_of_pax = raw_pax.replace("to", "-")
                            number_of_pax = number_of_pax.replace(" ", "")
            except:
                pass

            # Store structured message into dictionary
            message_data = {
                'user' : user,
                'post_date' : post_date,
                'post_time' : post_time,
                'raw_message' : raw_message,
                'user_type' : user_type,
                'pick_up' : pick_up,
                'drop_off' : drop_off,
                'request_date' : request_date,
                'request_time' : request_time,
                'number_of_pax' : number_of_pax
            }
            messages_list.append(message_data)

100%|████████████████████████████████████████████████████████████████████████████████| 405/405 [04:50<00:00,  1.19it/s]


### Stats

In [14]:
print("Total number of messages: \t\t", total_messages)
print("Total number of actual messages: \t", len(messages_list))
print()
print("Percentage of actual messages: \t", round((len(messages_list)/total_messages)*100, 2), "%")

Total number of messages: 		 365298
Total number of actual messages: 	 349801

Percentage of actual messages: 	 95.76 %


### Convert Data to CSV

In [15]:
data = pd.DataFrame(messages_list)
data.head()

,drop_off,number_of_pax,pick_up,post_date,post_time,raw_message,request_date,request_time,user,user_type
0,punggol,nil,alexandra/mbc/arc/vivo areas,26.02.2019,14:40:47,driver looking for hitchers<br/>pick up: alexa...,nil,nil,Stallion13,driver
1,bedok reservoir,nil,tuas area,26.02.2019,14:55:47,driver looking for hitchers<br/>pick up: tuas ...,nil,nil,xoxoxo,driver
2,off - ntu,nil,up - mountbatten mrt,26.02.2019,15:00:23,hitcher looking for drivers<br/>pick up - moun...,nil,nil,Amanda,hitcher
3,north-east,nil,bedok/tamp/pasir ris/eunos,26.02.2019,15:39:19,driver looking for hitchers<br/>pick up: bedok...,nil,nil,J,driver
4,north-east or east,nil,jurong,26.02.2019,18:12:47,driver looking for hitchers<br/>pick up: juron...,nil,nil,Colonel Ronald Silver,driver


In [16]:
data.to_csv("sg_hitch_data_updated.csv")